## Florida Department of Corrections Inmate Crime and Tattoo Database
#### Joey, Matt, Sean

### 1, 2. The Dataset, Downloaded

In [1]:
import pandas
import numpy as np
import pandas_access
DATASET = "./FDOC_January_2017.mdb" # From http://www.dc.state.fl.us/pub/obis_request.html 
                                    # ~1.3 gb; publicly available data.

### 3. Loading into a DF

In [2]:
dfs = {}
for tablename in pandas_access.list_tables(DATASET):
    dfs[tablename] = pandas_access.read_table(DATASET, tablename, low_memory = False)

### 4. Identifying Variables of Interest

In [3]:
dfs.keys()

[u'INMATE_RELEASE_INCARHIST',
 u'INMATE_ACTIVE_ROOT',
 u'INMATE_RELEASE_DETAINERS',
 u'OFFENDER_OFFENSES_CCS',
 u'INMATE_RELEASE_OFFENSES_CPS',
 u'INMATE_RELEASE_RESIDENCE',
 u'INMATE_RELEASE_SCARSMARKS',
 u'INMATE_ACTIVE_ALIASES',
 u'INMATE_ACTIVE_DETAINERS',
 u'INMATE_RELEASE_OFFENSES_prpr',
 u'INMATE_ACTIVE_SCARSMARKS',
 u'OFFENDER_ROOT',
 u'OFFENDER_RESIDENCE',
 u'INMATE_ACTIVE_OFFENSES_prpr',
 u'OFFENDER_ALIASES',
 u'INMATE_RELEASE_ROOT',
 u'INMATE_ACTIVE_INCARHIST',
 u'INMATE_ACTIVE_OFFENSES_CPS',
 u'CONTENTS',
 u'INMATE_RELEASE_ALIASES']

We initially planned to duplicate the experiment that linked tattoos and crimes, so we'll look at `SCARSMARKS` and `OFFENSES`.

### 5. Validation and Plotting

In [4]:
# http://www.dc.state.fl.us/pub/obis_request.html
# http://cdn.static-economist.com/sites/default/files/images/print-edition/20161224_XMC337_0.png

First, let's look at the number and types of crimes:

In [5]:
crimes = pandas.concat((dfs["INMATE_ACTIVE_OFFENSES_prpr"], dfs["INMATE_ACTIVE_OFFENSES_CPS"], dfs["INMATE_RELEASE_OFFENSES_prpr"], dfs["INMATE_RELEASE_OFFENSES_CPS"]))
crime_descriptions = crimes.adjudicationcharge_descr[crimes.adjudicationcharge_descr != "UNKNOWN"]
len(crime_descriptions)

2385210

In [6]:
crime_types = {
    "drugs": ["DRUG", "COCAINE", "SUBS", "MARIJUANA"],
    "theft": ["BURG", "THEFT", "THFT"],
    "robbery": ["ROBB"],
    "assault": ["ASLT", "ASSLT", "ASSAULT", "BATT"],
    "murder": ["MUR", "MRDR"]
}
for category, keywords in crime_types.items():
    count = crime_descriptions.str.contains("|".join(keywords)).value_counts()[True]
    print("{0: <7} {1: >6}".format(category, count))

drugs   427856
assault 206298
murder   33311
robbery 140578
theft   720649


Next, let's look at the number and types of tattoos:

In [7]:
tattoos = pandas.concat((dfs["INMATE_ACTIVE_SCARSMARKS"], dfs["INMATE_RELEASE_SCARSMARKS"]))
tattoo_descriptions = tattoos.Description
len(tattoo_descriptions)

1266840

In [8]:
tattoo_types = {
    "face": ["FACE", "HEAD", "CHEEK", "CHIN"],
    "white supremacist": ["WHITE", "ARYAN", "NAZI", "SWASTIKA", "14", "88"],
    "three dots": ["3 DOTS", "THREE DOTS", "THREEDOTS"],
    "tear drop": ["TEAR DROP", "TEARDROP"],
    "guns": ["GUN", "RIFLE", "BULLET"],
    "laugh now, cry later": ["LAUGH NOW", "CRY LATER", "LAUGHNOW", "CRYLATER"],
    "christian": ["CROSS", "HOLY", "ANGEL", "JESUS", "PRAY"],
    "satanic": ["PENTA", "DEVIL", "DEMON", "BAPHOMET"]
}
for category, keywords in tattoo_types.items():
    count = tattoo_descriptions.str.contains("|".join(keywords)).value_counts()[True]
    print("{0: <20} {1: >6}".format(category, count))

christian            138926
three dots             2753
satanic               18522
tear drop              7946
laugh now, cry later   4397
guns                  21472
white supremacist      6745
face                  71814


Now, let's group them together:

In [9]:
crimes_and_tattoos = pandas.merge(crimes, tattoos, "inner", "DCNumber")

In [10]:
grouped_crimes = crimes_and_tattoos.adjudicationcharge_descr.groupby(crimes_and_tattoos.DCNumber).apply(lambda x: ",".join(set(x.astype(str))))
grouped_tattoos = crimes_and_tattoos.Description.groupby(crimes_and_tattoos.DCNumber).apply(lambda x: ",".join(set(x.astype(str))))
crimes_and_tattoos = pandas.concat([grouped_crimes, grouped_tattoos], axis = 1).reset_index()

In [11]:
groupings = {}
for tattoo, tattoowords in tattoo_types.items():
    for crime, crimewords in crime_types.items():
        count = len(crimes_and_tattoos[crimes_and_tattoos.Description.str.contains("|".join(tattoowords)) &\
                          crimes_and_tattoos.adjudicationcharge_descr.str.contains("|".join(crimewords))])
        groupings[(tattoo, crime)] = count
        print("{0: <20} tattoos -> {1: <7} crimes: {2: >6} instances".format(tattoo, crime, count))

christian            tattoos -> drugs   crimes:  35032 instances
christian            tattoos -> assault crimes:  27839 instances
christian            tattoos -> murder  crimes:   4701 instances
christian            tattoos -> robbery crimes:  17438 instances
christian            tattoos -> theft   crimes:  46002 instances
three dots           tattoos -> drugs   crimes:    598 instances
three dots           tattoos -> assault crimes:    769 instances
three dots           tattoos -> murder  crimes:    156 instances
three dots           tattoos -> robbery crimes:    534 instances
three dots           tattoos -> theft   crimes:   1352 instances
satanic              tattoos -> drugs   crimes:   5001 instances
satanic              tattoos -> assault crimes:   4726 instances
satanic              tattoos -> murder  crimes:    842 instances
satanic              tattoos -> robbery crimes:   2736 instances
satanic              tattoos -> theft   crimes:   8018 instances
tear drop            tatt

And try to find a mapping:

In [12]:
# relative to crimes committed by same tattoo-havers
for tattoo in tattoo_types.keys():
    avg = sum([groupings[(tattoo, crime)]
               for crime in crime_types.keys()]) \
          / len(crime_types.keys())
    for crime in crime_types.keys():
        relation = "below" if groupings[(tattoo, crime)] < avg else "above"
        print("{0: <20} -> {1: <7}: {2: <5} average".format(tattoo, crime, relation))

christian            -> drugs  : above average
christian            -> assault: above average
christian            -> murder : below average
christian            -> robbery: below average
christian            -> theft  : above average
three dots           -> drugs  : below average
three dots           -> assault: above average
three dots           -> murder : below average
three dots           -> robbery: below average
three dots           -> theft  : above average
satanic              -> drugs  : above average
satanic              -> assault: above average
satanic              -> murder : below average
satanic              -> robbery: below average
satanic              -> theft  : above average
tear drop            -> drugs  : above average
tear drop            -> assault: above average
tear drop            -> murder : below average
tear drop            -> robbery: below average
tear drop            -> theft  : above average
laugh now, cry later -> drugs  : above average
laugh now, cr

In [13]:
# relative to tattoos had by same crime-committers
for crime in crime_types.keys():
    avg = sum([groupings[(tattoo, crime)]
               for tattoo in tattoo_types.keys()]) \
          / len(tattoo_types.keys())
    for tattoo in tattoo_types.keys():
        relation = "below" if groupings[(tattoo, crime)] < avg else "above"
        print("{0: <20} -> {1: <7}: {2: <5} average".format(tattoo, crime, relation))

christian            -> drugs  : above average
three dots           -> drugs  : below average
satanic              -> drugs  : below average
tear drop            -> drugs  : below average
laugh now, cry later -> drugs  : below average
guns                 -> drugs  : below average
white supremacist    -> drugs  : below average
face                 -> drugs  : above average
christian            -> assault: above average
three dots           -> assault: below average
satanic              -> assault: below average
tear drop            -> assault: below average
laugh now, cry later -> assault: below average
guns                 -> assault: below average
white supremacist    -> assault: below average
face                 -> assault: above average
christian            -> murder : above average
three dots           -> murder : below average
satanic              -> murder : below average
tear drop            -> murder : below average
laugh now, cry later -> murder : below average
guns         

### 6. Cleaning Issues

For validation, it's hard to know what the baseline that The Economist used was; are their relationships based on the typical crimes for someone with certain tattoos, or the typical tattoos had by someone who commits certain crimes, or something else entirely? We believe their actual baseline was inmates without tattoos, but it's hard to say.